## Combbine all diagnosis

This method take diagnosis from images, clinical, and diagnosis sheet, and creates one ground truth (where all three agree) and one majority vote (where two agree) diagnosis files. 

In [96]:
import pandas as pd
import math
clinical = pd.read_csv("ADSP_PHC_COGN.csv").rename(columns={"PHASE":"Phase"})
#this file is the metadata file that one can get from downloading MRI images from ADNI
img = pd.read_csv("metadata.csv")
comb = pd.read_csv("DXSUM_PDXCONV_ADNIALL.csv").rename(columns={"PHASE":"Phase"})
comb = comb[["RID", "PTID" , "Phase"]]

In [97]:
def read_diagnose(file_path: str = 'DXSUM_PDXCONV_ADNIALL.csv', verbose=False):
    # Read diagnostic summary
    diagnostic_summary = pd.read_csv(file_path, index_col='PTID').rename(columns={"PHASE":"Phase"})
    diagnostic_summary = diagnostic_summary.sort_values(by=["update_stamp"], ascending=True)
    # Create dictionary
    diagnostic_dict: dict = {}
    for key, data in diagnostic_summary.iterrows():
        # Iterate for each row of the document
        phase: str = data['Phase']
        diagnosis: float = -1.
        if phase == "ADNI1":
            diagnosis = data['DIAGNOSIS']
        elif phase == "ADNI2" or phase == "ADNIGO":
            dxchange = data['DIAGNOSIS']
            if dxchange == 1 or dxchange == 7 or dxchange == 9:
                diagnosis = 1.
            if dxchange == 2 or dxchange == 4 or dxchange == 8:
                diagnosis = 2.
            if dxchange == 3 or dxchange == 5 or dxchange == 6:
                diagnosis = 3.
        elif phase == "ADNI3":
            diagnosis = data['DIAGNOSIS']
        elif phase == "ADNI4":
            diagnosis = data['DIAGNOSIS']
        else:
            print(f"ERROR: Not recognized study phase {phase}")
            exit(1)
        # Update dictionary
        if not math.isnan(diagnosis):
            diagnostic_dict[key] = diagnosis
    if verbose:
        print_diagnostic_dict_summary(diagnostic_dict)
    return diagnostic_dict


def print_diagnostic_dict_summary(diagnostic_dict: dict):
    print(f"Number of diagnosed patients: {len(diagnostic_dict.items())}\n")
    n_NL = 0
    n_MCI = 0
    n_AD = 0
    for (key, data) in diagnostic_dict.items():
        if data == 1:
            n_NL += 1
        if data == 2:
            n_MCI += 1
        if data == 3:
            n_AD += 1
    print(f"Number of NL patients: {n_NL}\n"
          f"Number of MCI patients: {n_MCI}\n"
          f"Number of AD patients: {n_AD}\n")

In [98]:
d = read_diagnose()
print_diagnostic_dict_summary(d)

Number of diagnosed patients: 3227

Number of NL patients: 1225
Number of MCI patients: 1069
Number of AD patients: 933



In [99]:
new = pd.DataFrame.from_dict(d, orient='index').reset_index()
print(new)

            index    0
0      011_S_0002  2.0
1      011_S_0003  3.0
2      011_S_0005  1.0
3      011_S_0008  2.0
4      022_S_0007  3.0
...           ...  ...
3222  016_S_10324  1.0
3223  114_S_10321  1.0
3224  123_S_10292  2.0
3225  035_S_10329  3.0
3226  082_S_10219  3.0

[3227 rows x 2 columns]


In [100]:
clinical.head()

,RID,SUBJECT_KEY,Phase,VISCODE,VISCODE2,EXAMDATE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,AGE,DX,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP,update_stamp
0,2,ADNI_011_S_0002,ADNI2,v06,m72,2011-09-19,1,16.0,2,5,80.4682,1.0,0.090,0.002,0.460,0.264,2021-04-14 13:05:55.0
1,2,ADNI_011_S_0002,ADNI2,v21,m96,2013-09-09,1,16.0,2,5,82.4422,1.0,0.288,-0.495,0.460,NaN,2021-04-14 13:05:56.0
2,2,ADNI_011_S_0002,ADNI1,m36,m36,2008-08-27,1,16.0,2,5,77.4073,1.0,0.344,0.419,0.535,NaN,2021-04-14 13:05:56.0
3,2,ADNI_011_S_0002,ADNI2,v41,m120,2015-09-22,1,16.0,2,5,84.4764,1.0,0.382,-0.549,0.595,-0.333,2021-04-14 13:05:56.0
4,2,ADNI_011_S_0002,ADNIGO,m60,m60,2010-09-22,1,16.0,2,5,79.4771,1.0,0.101,0.066,0.620,0.264,2021-04-14 13:05:56.0


In [101]:
clinical["year"] = clinical["EXAMDATE"].str[:4]

In [102]:
clinical["Subject"] = clinical["SUBJECT_KEY"].str.replace("ADNI_", "").str.replace("s", "S")

In [103]:
c = comb.merge(clinical, on = ["RID", "Phase"])

In [104]:
c = c.drop("Subject", axis =1)

In [105]:
c = c.rename(columns = {"PTID":"Subject"})

In [106]:
img["year"] = img["EXAMDATE"].str[5:].str.replace("/", "")

In [107]:
img = img.replace(["CN", "MCI", "AD"], [ 0, 1, 2])

In [108]:
c["DX"] = c["DX"] -1

In [109]:
new[0] = new[0].astype(int) -1

In [110]:
new = new.rename(columns = {"index":"Subject", 0:"GroupN"})

In [111]:
img = img.rename(columns = {"PTID":"Subject"})

In [112]:
m = new.merge(c, on = "Subject", how = "outer").merge(img, on = "Subject", how = "outer")
print(m)
for i in m:
    print(i)

           Subject  GroupN  RID_x  Phase SUBJECT_KEY VISCODE_x VISCODE2_x  \
0       002_S_0295     0.0  295.0  ADNI1         NaN        bl         bl   
1       002_S_0295     0.0  295.0  ADNI1         NaN        bl         bl   
2       002_S_0295     0.0  295.0  ADNI1         NaN       m06        m06   
3       002_S_0295     0.0  295.0  ADNI1         NaN       m06        m06   
4       002_S_0295     0.0  295.0  ADNI1         NaN       m12        m12   
...            ...     ...    ...    ...         ...       ...        ...   
227139  941_S_7074     0.0    NaN    NaN         NaN       NaN        NaN   
227140  941_S_7085     1.0    NaN    NaN         NaN       NaN        NaN   
227141  941_S_7087     0.0    NaN    NaN         NaN       NaN        NaN   
227142  941_S_7091     0.0    NaN    NaN         NaN       NaN        NaN   
227143  941_S_7106     1.0    NaN    NaN         NaN       NaN        NaN   

        EXAMDATE_x  PTGENDER  PTEDUCAT  ...  FLDSTRNGTH        ID  SITEID  

In [113]:
m[["GroupN", "DX", "Group"]]

KeyError: "['Group'] not in index"

In [ ]:
m = m[["Subject", "GroupN", "Group", "DX", "Phase"]].drop_duplicates()

In [ ]:
m = m.dropna(subset = ["GroupN", "Group", "DX"], how="all").drop_duplicates()
m

In [ ]:
m.loc[m["DX"].isna() & m["Group"].isna(), "Group"] = m.loc[m["DX"].isna() & m["Group"].isna(), "GroupN"]
m.loc[m["DX"].isna() & m["Group"].isna(), "DX"] = m.loc[m["DX"].isna() & m["Group"].isna(), "GroupN"]

In [ ]:
m1 = m[m["GroupN"] == m["Group"]]
m3 = m[m["GroupN"] == m["DX"]]
m4 = m[m["Group"] == m["DX"]]
m2 = m1[m1["Group"] == m1["DX"]]

In [ ]:
m1 = m1[["Subject", "GroupN", "Group", "DX", "Phase"]]
m1

In [ ]:
m1.loc[m1["DX"].isna(), "DX"] = m1.loc[m1["DX"].isna(), "Group"]

In [ ]:
m3 = m3[["Subject", "GroupN", "Group", "DX", "Phase"]]
m3

In [ ]:
m3.loc[m3["Group"].isna(), "Group"] = m3.loc[m3["Group"].isna(), "GroupN"]

In [ ]:
m4 = m4[["Subject", "GroupN", "Group", "DX", "Phase"]]
m4

In [ ]:
m4[m4["GroupN"] != m4["DX"]]

In [ ]:
m2[["Subject", "GroupN", "Group", "DX", "Phase"]]

In [ ]:
m5 = pd.concat([m1,m3,m4])
i = m5[m5["Group"] == m5["GroupN"]]
i = i[i["Group"] == i["DX"]]

In [ ]:
i = i.drop_duplicates()

In [ ]:
i

In [ ]:
i[["Subject", "Group", "Phase"]].to_csv("ground_truth.csv")

In [ ]:
m.update(m5[~m5.index.duplicated(keep='first')])

In [ ]:
indexes = m.index

In [ ]:
#if none of the three diagnosis agree, then we set the value to -1
m["GROUP"] = -1

In [ ]:
for i in indexes:
    row = m.loc[i]
    if (row["GroupN"] == row["Group"]):
        val = row["GroupN"]
        
        m.loc[i, "GROUP"] = val
    elif (row["GroupN"] == row["DX"]):
        val = row["GroupN"]
        m.loc[i, "GROUP"] = val
        
    elif (row["Group"] == row["DX"]):
        val = row["Group"]
        m.loc[i, "GROUP"] = val
        

In [ ]:
m5 = m5[~m5.index.duplicated(keep='first')]
m5

In [ ]:
m[m["GROUP"] != -1]

In [ ]:
m[["Subject", "GroupN", "Group", "DX", "GROUP", "Phase"]].to_csv("diagnosis_full.csv")